<h1>Introduction

<h3>A description of the problem and a discussion of the background. (15 marks)</h3>
As a person who enjoys visiting parks and plazas, I want to live in a zip code with more parks in plazas. I prefer the cleaner air these spaces provide as well as the green with the grass and the trees. I enjoy listening to birds and playing in the park. Any chance to escape the city a bit is a chance I try to take.

<h3>A description of the data and how it will be used to solve the problem. (15 marks)</h3>
I will be using foursquare's api to pull data on parks around the city of Madrid such as their name and geo coordinates. These parks will be compared with zip codes within a walking distance of 1 km. All zip codes and parks will be placed on the map. Zip code points will be colored based on the number of parks that are contained within.

<h1>Data

<h3>This project begins by downloading all necessary dependencies

In [26]:
import requests
import urllib.request
import lxml.html as lh
import pandas as pd
import json
from pandas.io.json import json_normalize
import numpy as np
import geopy
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from apitest import *
print("Libraries imported")

Libraries imported


<h3>Reading the file with zip codes and their geocoordinates

In [27]:
df = pd.read_csv('madrid_zip_codes_latlon.csv')
df.head()

,Zip_Code,Latitude,Longitude
0,28001,40.424549,-3.68419
1,28002,40.449268,-3.67406
2,28003,40.443001,-3.69812
3,28004,40.424759,-3.69410
4,28005,40.404720,-3.71615


<h3>Determining the coordinates of Madrid

In [28]:
madrid_address = 'Madrid, Madrid'
geolocator = Nominatim(user_agent="madrid_explore")
location = geolocator.geocode(madrid_address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Madrid are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Madrid are 40.4167047, -3.7035825.


<H3>This map showcases the location of each of the zip codes within Madrid

In [29]:
# create map
intro_map = folium.Map(location=[latitude, longitude], zoom_start=12)



# add markers to the map
markers_colors = []
for lat, lon, poi in zip(df['Latitude'], df['Longitude'], df['Zip_Code']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(intro_map)
       
intro_map

<h3> Code to clean up the map zip codes, so they are more condensed and closer to the city

In [30]:
# Get names of indexes for which column category is a far zipcode
drops = df[df['Zip_Code'] == 28024].index
#Must remove 28070, 28048, 28042, 28052, 28051, 28021, 28054, 28044, 28024 
# Delete these row indexes from dataFrame
df.drop(drops , inplace=True)

In [31]:
df.shape

(56, 3)

<h1> API Credentials

In [32]:
CLIENT_ID = foursquareid
CLIENT_SECRET = foursquaresecret
VERSION = '20200124'
radius = 1000
LIMIT = 500
CATEGORY = '4bf58dd8d48988d163941735'  #category for parks in foursquare API

<H2>Main Function Call
    <h3>Gets data on how many parks are found in each of the zip codes' coordinates

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)  - commented out once I got this working properly
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore/?&categoryId={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CATEGORY,
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip Code', 
                  'Zip Code Latitude', 
                  'Zip Code Longitude', 
                  'Park Name', 
                  'Park Latitude', 
                  'Park Longitude', 
                  'Park Category']
    
    return(nearby_venues)

In [34]:
df_counts = getNearbyVenues(names=df['Zip_Code'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

<h1>Methodology</h1>
Once we have the data, we can now see how many parks are within walking distance of the zip codes.  We also have the names of the parks.  From here we preview the names of the parks and determine the number of parks per zip code.
After that, we run a k means analysis which clusters the zip codes based on the availability of parks within that zip code.  We generate a new table with the k means information and merge that with the previous table to get our final df_merged table with all the data necessary to produce the map.

<h2>Determine the size of our sample.  I am not sure why I could not get more than 107 to show up.  This took many hours of playing before moving on

In [35]:
print(df_counts.shape)
df_counts

(107, 7)


,Zip Code,Zip Code Latitude,Zip Code Longitude,Park Name,Park Latitude,Park Longitude,Park Category
0,28001,40.424549,-3.68419,Puerta de Madrid de los Jardines del Buen Retiro,40.420795,-3.681858,Park
1,28001,40.424549,-3.68419,Puerta de O'Donnell de los Jardines del Buen R...,40.421506,-3.680121,Park
2,28002,40.449268,-3.67406,Parque de Berlín,40.450409,-3.675699,Park
3,28002,40.449268,-3.67406,Parque Del Nieremberg,40.448780,-3.673347,Park
4,28002,40.449268,-3.67406,Parque Calle De La Condesa De Santamarca,40.449640,-3.672254,Park
...,...,...,...,...,...,...,...
102,28050,40.500500,-3.66740,Parque San Juan de Ortega / Puente de la Reina,40.501044,-3.670950,Park
103,28054,40.362800,-3.75620,Parque La Peseta,40.364057,-3.756723,Park
104,28054,40.362800,-3.75620,Parque Casa Milà,40.360333,-3.760752,Park
105,28071,40.426500,-3.69160,Parque Villa de Paris,40.426631,-3.693712,Park


In [36]:
pd.unique(df_counts['Park Name'])

array(['Puerta de Madrid de los Jardines del Buen Retiro',
       "Puerta de O'Donnell de los Jardines del Buen Retiro",
       'Parque de Berlín', 'Parque Del Nieremberg',
       'Parque Calle De La Condesa De Santamarca',
       'El Mejor Curro De La Primavera', 'Parc de la Ciutadella',
       'Canal de Isabel II', 'Parque Ríos Rosas', 'Parque Villa de Paris',
       'Parque de los mendigos', 'Plaza del Olivo',
       'Jardines de Concha Piquer', 'Parque de la Ventosa',
       'Parque Nuevos Ministerios', 'Little Manhattan', 'parque la red',
       'Parque Martin Luther King', 'Parque del Oeste',
       'Paseo del Pintor Rosales', 'Parque del Retiro',
       'Palacio de Cristal del Retiro', 'Parque de Madrid',
       'Bosque del Recuerdo',
       'Puerta de España de los Jardines del Buen Retiro',
       'Monumento a Alfonso XII de España', 'Paseo de las Estatuas',
       'Puerta de Murillo de los Jardines del Buen Retiro',
       'Puerta de Felipe IV', 'Estanque del Retiro', 'Xúquer

<h3>Determine how many parks are found within the zip code

In [37]:
df_final = df_counts.groupby('Zip Code')['Park Name'].nunique()
df_final = pd.DataFrame(df_final)
df_final.reset_index()

,Zip Code,Park Name
0,28001,2
1,28002,4
2,28003,3
3,28004,3
4,28005,2
5,28006,2
6,28007,2
7,28008,2
8,28009,10
9,28010,2


<h1> Cluster Zip Codes based on availability of parks

In [38]:
# set number of clusters
kclusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:] 

array([1, 0, 3, 3, 1, 1, 1, 1, 2, 1, 3, 2, 1, 1, 3, 1, 1, 1, 1, 1, 3, 3,
       0, 3, 1, 1, 0, 3, 1, 1, 0, 0, 1, 3, 1, 1, 1, 1])

In [39]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=4, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [40]:
df_final.insert(0, 'Cluster Labels', kmeans.labels_)

In [41]:
df_final.rename(columns={"Park Name": "Park Count"}, inplace=True)

<h3>Reset the index before joining the dataframes for the final map

In [42]:
df_final.reset_index()

,Zip Code,Cluster Labels,Park Count
0,28001,1,2
1,28002,0,4
2,28003,3,3
3,28004,3,3
4,28005,1,2
5,28006,1,2
6,28007,1,2
7,28008,1,2
8,28009,2,10
9,28010,1,2


<h1> Time to join the columns

In [43]:
df_merged = df_counts.join(df_final, on='Zip Code')

df_merged.head() # check the last columns!

,Zip Code,Zip Code Latitude,Zip Code Longitude,Park Name,Park Latitude,Park Longitude,Park Category,Cluster Labels,Park Count
0,28001,40.424549,-3.68419,Puerta de Madrid de los Jardines del Buen Retiro,40.420795,-3.681858,Park,1,2
1,28001,40.424549,-3.68419,Puerta de O'Donnell de los Jardines del Buen R...,40.421506,-3.680121,Park,1,2
2,28002,40.449268,-3.67406,Parque de Berlín,40.450409,-3.675699,Park,0,4
3,28002,40.449268,-3.67406,Parque Del Nieremberg,40.448780,-3.673347,Park,0,4
4,28002,40.449268,-3.67406,Parque Calle De La Condesa De Santamarca,40.449640,-3.672254,Park,0,4


<h1>Results</h1>
Final map display to show the zip codes with color based on the number of parks.  The black spots are parks or plazas.

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set colors for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# adding markers to the map
markers_colors = []
for lat, lon, poi, cluster, parks in zip(df_merged['Zip Code Latitude'], 
                                         df_merged['Zip Code Longitude'], 
                                         df_merged['Zip Code'], 
                                         df_merged['Cluster Labels'],
                                         df_merged['Park Count']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster) + "\nThere are " + str(parks) + " parks.", parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=8,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# adding parks to the map
for lat, lon, name in zip(df_merged['Park Latitude'],
                          df_merged['Park Longitude'],
                          df_merged['Park Name']):
    label2 = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
    [lat, lon],
    radius=3,
    popup=label2,
    color='black',
    fill=True).add_to(map_clusters)

       
map_clusters

<h1>Discussion</h1>
When looking at the map you can see that there are other parks visible that the foursquare api did not pick up.  I've looked through the documentation but have been unable to determine if they are a part of foursquare, or if I'd limited the api call in some way.  Either way this is a good start to analayzing zip codes and their availability of parks.  I would like to run this analaysis again trying other location api companies to see how it differs in quality and information.  For now, though it looks like there are two prime zip codes to live in for lots of green space.  These are 28013 and 28009 each with a count of 10 parks or plazas that were pulled from Foursquare's api.

<h1>Conclusion</h1>
Thank you for taking the time to look at this. This project is the very first of many data science projects that I will be working on in the future.  It brought me great pleasure to produce this.  I know that from here they will only get better.  This is definitely a journey of learning.